# Simple Entry Removal Algorithm 1 (SER 1)

In [2]:
import numpy as np
import datetime
import pandas as pd
import json

### Description of add_up_a_decomposition
The function takes as inputs:
1. a decomposition (x_list, BN_matrices_array) output by SER 1, SER 2, GER (x_list is a list, whereas BN_matrices_array is a 2D np array).
2. The dimension of the output TPM row_dim, col_dim

The function then sums up (x_list, BN_matrices_array) and outputs the resulting TPM (row_dim-by-col_dim).

In [3]:
def add_up_a_decomposition(x_list, BN_matrices_array, row_dim, col_dim):
    output_TPM = np.zeros((row_dim, col_dim), dtype=int)
    num_BN_matrices = len(x_list)
    
    for k in range(num_BN_matrices):
        for col in range(col_dim):
            output_TPM[BN_matrices_array[k, col], col] += x_list[k]
    
    return output_TPM

# The SER 1 algorithm

In [4]:
# This function is used in the SER 1 function.
# residue_matrix_R is a np.ndarray.
# PBN_row_num is the number of rows present in residue_matrix_R.
# PBN_col_num is the number of columns present in residue_matrix_R.
def find_smallest_nonzero_entry(residue_matrix_R, PBN_row_num, PBN_col_num):
    row_counter = 0
    col_counter = 0
    while residue_matrix_R[row_counter, col_counter] == 0:
        row_counter += 1
    smallest_entry = residue_matrix_R[row_counter, col_counter]
    smallest_entry_row_pos = row_counter
    smallest_entry_col_pos = col_counter

    del row_counter
    del col_counter
    for col_counter in range(PBN_col_num):
        for row_counter in range(PBN_row_num):
            current_entry = residue_matrix_R[row_counter, col_counter]
            if (current_entry > 0) and (current_entry < smallest_entry):
                smallest_entry = current_entry
                smallest_entry_row_pos = row_counter
                smallest_entry_col_pos = col_counter

    return smallest_entry, smallest_entry_row_pos, smallest_entry_col_pos

In [5]:
def SER_1(PBN_matrix_P):  # input: 2^n-by-2^n ndarray consisting of integers.
    PBN_row_num = PBN_matrix_P.shape[0]
    PBN_col_num = PBN_matrix_P.shape[1]
    coefficient_list_xi = []  # the coefficients of BN matrices to be output
    # the BN matrices to be output
    BN_matrices_list_Ai = np.array([[-1] * PBN_col_num])  # the first row of BN_matrices_list_Ai will be
                                                          # replaced in the first iteration of pseudocode
                                                          # steps 1 to 4.
                                                          
    residue_matrix_R = PBN_matrix_P.copy()
    k = 1  # counts the number of iterations of pseudocode Steps 1 to 4
    chosen_entry_xi, xi_row_pos, xi_col_pos = find_smallest_nonzero_entry(residue_matrix_R, 
                                                                          PBN_row_num, PBN_col_num)
    chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
    chosen_entry_pos_each_col[0, xi_col_pos] = xi_row_pos
    
    coefficient_list_xi.append(chosen_entry_xi)
    BN_matrices_list_Ai = chosen_entry_pos_each_col
    
    for col_counter in range(PBN_col_num):
        row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
        residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    zero_PBN_matrix = np.zeros((PBN_row_num, PBN_col_num), dtype=int)
    
    while (not np.array_equal(residue_matrix_R, zero_PBN_matrix)):
        k += 1
        chosen_entry_xi, xi_row_pos, xi_col_pos = find_smallest_nonzero_entry(residue_matrix_R, 
                                                                              PBN_row_num, PBN_col_num)
        chosen_entry_pos_each_col = np.argmax(residue_matrix_R, axis=0, keepdims=True)
        chosen_entry_pos_each_col[0, xi_col_pos] = xi_row_pos
    
        coefficient_list_xi.append(chosen_entry_xi)
        BN_matrices_list_Ai = np.append(BN_matrices_list_Ai, chosen_entry_pos_each_col, axis=0)
        
        for col_counter in range(PBN_col_num):
            row_to_deduct = chosen_entry_pos_each_col[0, col_counter]
            residue_matrix_R[row_to_deduct, col_counter] -= chosen_entry_xi
    
    return k, coefficient_list_xi, BN_matrices_list_Ai

# "Truncated Floating-Point Matrix" Experiment

In [6]:
# Custom JSON encoder that handles NumPy types (created by GPT-o3 on poe.com)
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)):
            return int(obj)
        elif isinstance(obj, (np.floating,)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

## $8 \times 8$ 5300-TPMs

In [8]:
for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './8-by-8 (trunc)/dim8_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom = SER_1(tTPM)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (trunc_SER_1)/dim8_decom_m" + str(k) + ".json"
    time_filepath = "./raw data (trunc_SER_1)/dim8_time_m" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 16:27:00.741097

iteration 1
Time now is: 2025-04-12 16:27:00.748615

iteration 2
Time now is: 2025-04-12 16:27:00.768161

iteration 3
Time now is: 2025-04-12 16:27:00.783507

iteration 4
Time now is: 2025-04-12 16:27:00.800276

iteration 5
Time now is: 2025-04-12 16:27:00.840220

iteration 6
Time now is: 2025-04-12 16:27:00.858268

iteration 7
Time now is: 2025-04-12 16:27:00.879362

iteration 8
Time now is: 2025-04-12 16:27:00.895796

iteration 9
Time now is: 2025-04-12 16:27:00.948477



## $16 \times 16$ 5300-TPMs

In [9]:
for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './16-by-16 (trunc)/dim16_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom = SER_1(tTPM)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (trunc_SER_1)/dim16_decom_m" + str(k) + ".json"
    time_filepath = "./raw data (trunc_SER_1)/dim16_time_m" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 16:30:51.898712

iteration 1
Time now is: 2025-04-12 16:30:51.973668

iteration 2
Time now is: 2025-04-12 16:30:52.011151

iteration 3
Time now is: 2025-04-12 16:30:52.051558

iteration 4
Time now is: 2025-04-12 16:30:52.088654

iteration 5
Time now is: 2025-04-12 16:30:52.128443

iteration 6
Time now is: 2025-04-12 16:30:52.164463

iteration 7
Time now is: 2025-04-12 16:30:52.207408

iteration 8
Time now is: 2025-04-12 16:30:52.245543

iteration 9
Time now is: 2025-04-12 16:30:52.288617



## $32 \times 32$ 5300-TPMs

In [13]:
for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './32-by-32 (trunc)/dim32_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom = SER_1(tTPM)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (trunc_SER_1)/dim32_decom_m" + str(k) + ".json"
    time_filepath = "./raw data (trunc_SER_1)/dim32_time_m" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 16:35:35.326601

iteration 1
Time now is: 2025-04-12 16:35:35.566691

iteration 2
Time now is: 2025-04-12 16:35:35.807982

iteration 3
Time now is: 2025-04-12 16:35:35.974069

iteration 4
Time now is: 2025-04-12 16:35:36.173500

iteration 5
Time now is: 2025-04-12 16:35:36.422888

iteration 6
Time now is: 2025-04-12 16:35:36.568472

iteration 7
Time now is: 2025-04-12 16:35:36.765959

iteration 8
Time now is: 2025-04-12 16:35:36.971952

iteration 9
Time now is: 2025-04-12 16:35:37.198692



## $64 \times 64$ 5300-TPMs

In [17]:
for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './64-by-64 (trunc)/dim64_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    start_time = datetime.datetime.now()
    decom = SER_1(tTPM)
    end_time = datetime.datetime.now()
    execution_duration = end_time - start_time
    
    exe_dur_seconds = execution_duration.total_seconds()
    exe_dur_dict = {"execution time": exe_dur_seconds}
    
    # save the data
    decom_filepath = "./raw data (trunc_SER_1)/dim64_decom_m" + str(k) + ".json"
    time_filepath = "./raw data (trunc_SER_1)/dim64_time_m" + str(k) + ".json"
    
    with open(decom_filepath, 'w') as out1:
        json.dump(decom, out1, cls=NumpyEncoder)
        
    with open(time_filepath, 'w') as out2:
        json.dump(exe_dur_dict, out2)
    
    print()

iteration 0
Time now is: 2025-04-12 16:39:45.222324

iteration 1
Time now is: 2025-04-12 16:39:46.540898

iteration 2
Time now is: 2025-04-12 16:39:47.634243

iteration 3
Time now is: 2025-04-12 16:39:48.921757

iteration 4
Time now is: 2025-04-12 16:39:50.085126

iteration 5
Time now is: 2025-04-12 16:39:51.328300

iteration 6
Time now is: 2025-04-12 16:39:52.540342

iteration 7
Time now is: 2025-04-12 16:39:53.812084

iteration 8
Time now is: 2025-04-12 16:39:55.029567

iteration 9
Time now is: 2025-04-12 16:39:56.276504

